In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten

Using TensorFlow backend.


In [44]:
# Run if using tensorflow2.0+
import math
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df_train = pd.read_csv('../Data/train.csv')
df_train

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0
...,...,...,...
4999995,499.9996,2.9194,7
4999996,499.9997,2.6980,7
4999997,499.9998,4.5164,8
4999998,499.9999,5.6397,9


In [51]:
def get_data(batch_num='all', zeros = True, randomize=True):
    #create training data per batch
    batch_size = 500000
    if batch_num == 'all':
        beg = 0
        end = 499999999
        print('Training on all data')
    else:
        batch = batch_num
        beg = (batch-1)*batch_size 
        end = batch*batch_size - 1
    df_batch = df_train[beg:end]
    # filter out the zero open_channels cases
    if not zeros:
        df_batch = df_batch.drop(df_batch[df_batch['open_channels']==0].index) #optional, added because model was always predicting 0
        
    # randomize 
    signal = np.array(df_batch.signal)
    open_channels = np.array(df_batch.open_channels)
    if randomize:
        x_train, x_test, y_train, y_test = train_test_split(signal,open_channels,test_size=0.25)
        x_train = np.reshape(x_train, (-1,1))
        y_train.reshape(-1,1)
        x_test = np.reshape(x_test, (-1,1))
        y_test.reshape(-1,1)
    
    return x_train, x_test, y_train, y_test

In [70]:
acc = []
for batch in range(1,11):    
    x_train,x_test, y_train,y_test = get_data(batch,zeros=False)
    reg = RandomForestRegressor(n_estimators=50)
    reg = reg.fit(x_train,y_train)
    y_pred = reg.predict(x_test)
    rounded = y_pred.round().astype(int)
    score = accuracy_score(y_test, rounded)
    acc.append(score)
acc

[1.0,
 1.0,
 1.0,
 0.9827865897620764,
 0.861088,
 0.9679306834667885,
 1.0,
 0.5646541468737437,
 0.4784142099103865,
 0.384912]

In [22]:
predictions = rounded

In [32]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = round(y[i][j][0])
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
        elif actual == 0 and pred > 0:
            fp += 1
        elif actual > 0 and pred == 0:
            fn += 1
        else:
            tp += 1
total = tp+tn+fp+fn

(3692391, 1044351, 195801, 67457)

In [34]:
print("Accuracy:",(tp+tn)*100/total)
print("True Positive Rate:",tp*100/total)
print("True Negative Rate:",tn*100/total)
print("False Postiive Rate:",fp*100/total)
print("False Negative Rate:",fn*100/total)

Accuracy: 94.73484
True Positive Rate: 73.84782
True Negative Rate: 20.88702
False Postiive Rate: 3.91602
False Negative Rate: 1.34914


In [38]:
p_error = 0
n_error = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = y[i][j][0]
        pred = predictions[i][j][0]
        if round(actual) == 0:
            n_error += abs(actual-pred)
        else:
            p_error += abs(actual-pred)

In [39]:
print("Average Error:",(p_error+n_error)/total)
print("Actual Positive Average Error:",p_error/total)
print("Actual Negative Average Error:",n_error/total)

Average Error: 1.320657059439701
Actual Positive Average Error: 1.2318114114205538
Actual Negative Average Error: 0.08884564801914692
